In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import logging
from horgues3.dataset import HorguesDataset
from horgues3.models import HorguesModel, PlackettLuceLoss, WeightedPlackettLuceLoss, ListwiseLoss, PairwiseRankingLoss, CombinedRankingLoss, RankNetLoss
from horgues3.betting import calculate_betting_probabilities, format_betting_results
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns


# ログ設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def evaluate_betting_probabilities(model, dataloader, device, num_samples=5):
    """モデルの予測から馬券確率を計算して表示"""
    
    model.eval()
    sample_count = 0
    
    with torch.no_grad():
        for batch in dataloader:
            if sample_count >= num_samples:
                break
            
            # 残り必要なサンプル数を計算
            remaining_samples = num_samples - sample_count
            batch_size = batch['mask'].size(0)
            
            # バッチサイズが残り必要数を超える場合はスライス
            if batch_size > remaining_samples:
                # 各要素をスライス
                x_num = {k: v[:remaining_samples] for k, v in batch['x_num'].items()}
                x_cat = {k: v[:remaining_samples] for k, v in batch['x_cat'].items()}
                mask = batch['mask'][:remaining_samples]
                race_ids = batch['race_id'][:remaining_samples]
            else:
                x_num = batch['x_num']
                x_cat = batch['x_cat']
                mask = batch['mask']
                race_ids = batch['race_id']
            
            # データをデバイスに移動
            x_num = {k: v.to(device) for k, v in x_num.items()}
            x_cat = {k: v.to(device) for k, v in x_cat.items()}
            mask = mask.to(device)
            
            # 予測
            scores = model(x_num=x_num, x_cat=x_cat, mask=mask)
            
            # CPUに移動してnumpy配列に変換
            scores_np = scores.cpu().numpy()
            mask_np = mask.cpu().numpy()
            
            # 馬券確率を計算
            probabilities = calculate_betting_probabilities(
                horse_strengths=scores_np,
                mask=mask_np,
                temperature=1.0
            )
            
            # 結果をフォーマットして表示
            results = format_betting_results(
                race_ids=race_ids,
                probabilities=probabilities,
                masks=mask_np
            )
            
            print(results)
            
            # 実際に処理したサンプル数を更新
            current_batch_size = mask.size(0)
            sample_count += current_batch_size
            
            # 目標サンプル数に達したら終了
            if sample_count >= num_samples:
                break

In [ ]:
def plot_scores_distribution(scores, mask, phase="Train"):
    """マスクされた有効な部分のスコア分布を表示"""
    # マスクが1の部分のスコアのみを取得
    valid_scores = []
    for batch_idx in range(scores.shape[0]):
        for horse_idx in range(scores.shape[1]):
            if mask[batch_idx, horse_idx] == 1:
                valid_scores.append(scores[batch_idx, horse_idx])
    
    valid_scores = np.array(valid_scores)
    
    # 分布の統計情報
    stats = {
        'mean': np.mean(valid_scores),
        'std': np.std(valid_scores),
        'min': np.min(valid_scores),
        'max': np.max(valid_scores),
        'count': len(valid_scores)
    }
    
    # プロット
    plt.figure(figsize=(10, 6))
    
    # ヒストグラム
    plt.subplot(1, 2, 1)
    plt.hist(valid_scores, bins=50, alpha=0.7, edgecolor='black')
    plt.title(f'{phase} Scores Distribution (Valid Horses Only)')
    plt.xlabel('Score Value')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    
    # 統計情報をテキストで表示
    stats_text = f'Count: {stats["count"]}\nMean: {stats["mean"]:.4f}\nStd: {stats["std"]:.4f}\nMin: {stats["min"]:.4f}\nMax: {stats["max"]:.4f}'
    plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # ボックスプロット
    plt.subplot(1, 2, 2)
    plt.boxplot(valid_scores, vert=True)
    plt.title(f'{phase} Scores Box Plot')
    plt.ylabel('Score Value')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return stats

def train_epoch(model, dataloader, criterion, optimizer, device):
    """1エポックの学習（勾配監視とクリッピング調整）"""
    model.train()
    total_loss = 0
    num_batches = 0
    all_scores = []
    all_masks = []
    grad_norms = []
    
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        
        # データをデバイスに移動
        x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
        x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
        rankings = batch['rankings'].to(device)
        mask = batch['mask'].to(device)
        
        # 予測
        scores = model(x_num=x_num, x_cat=x_cat, mask=mask)
        
        # 損失計算
        loss = criterion(scores, rankings, mask)
        
        # 逆伝播
        loss.backward()
        
        # 勾配ノルムを記録
        total_grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        grad_norms.append(total_grad_norm.item())
        
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
        
        # スコア分析用にデータを保存（最初の数バッチのみ）
        if len(all_scores) < 5:
            all_scores.append(scores.detach().cpu().numpy())
            all_masks.append(mask.detach().cpu().numpy())
    
    # 勾配ノルムの統計を表示
    if grad_norms:
        print(f"Grad norm - Mean: {np.mean(grad_norms):.4f}, Max: {np.max(grad_norms):.4f}, Min: {np.min(grad_norms):.4f}")
    
    # スコア分布を表示
    if all_scores:
        combined_scores = np.concatenate(all_scores, axis=0)
        combined_masks = np.concatenate(all_masks, axis=0)
        train_stats = plot_scores_distribution(combined_scores, combined_masks, "Train")
        print(f"Train Scores Stats: {train_stats}")
    
    return total_loss / num_batches if num_batches > 0 else 0

def validate_epoch(model, dataloader, criterion, device):
    """1エポックの検証（スコア分析付き）"""
    model.eval()
    total_loss = 0
    num_batches = 0
    all_scores = []
    all_masks = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            # データをデバイスに移動
            x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
            x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
            rankings = batch['rankings'].to(device)
            mask = batch['mask'].to(device)
            
            # 予測
            scores = model(x_num=x_num, x_cat=x_cat, mask=mask)
            
            # 損失計算
            loss = criterion(scores, rankings, mask)
            
            total_loss += loss.item()
            num_batches += 1
            
            # スコア分析用にデータを保存（最初の数バッチのみ）
            if len(all_scores) < 5:  # 最初の5バッチのみ
                all_scores.append(scores.detach().cpu().numpy())
                all_masks.append(mask.detach().cpu().numpy())
    
    # スコア分布を表示
    if all_scores:
        combined_scores = np.concatenate(all_scores, axis=0)
        combined_masks = np.concatenate(all_masks, axis=0)
        val_stats = plot_scores_distribution(combined_scores, combined_masks, "Validation")
        print(f"Validation Scores Stats: {val_stats}")
    
    return total_loss / num_batches if num_batches > 0 else 0

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logger.info(f"Using device: {device}")

# データセット準備
logger.info("Preparing dataset...")

# 学習データの準備
logger.info("Creating new preprocessors...")
train_dataset = HorguesDataset(max_horses=18)
train_dataset.fetch('20220131', '20221231').process().fit().transform().build_races()

# 検証データの準備
val_dataset = HorguesDataset(max_horses=18)
val_dataset.set_preprocessors(train_dataset.get_preprocessors())
val_dataset.fetch('20230105', '20230105').process().transform().build_races()

logger.info(f"Training samples: {len(train_dataset)}")
logger.info(f"Validation samples: {len(val_dataset)}")

# データローダー
train_loader = DataLoader(
    train_dataset, 
    batch_size=32, 
    shuffle=True, 
    num_workers=0  # Windowsでは0に設定
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=32, 
    shuffle=False, 
    num_workers=0
)

# モデル設定
feature_configs = train_dataset.get_feature_configs()
logger.info(f"Feature configs: {feature_configs}")

model = HorguesModel(**feature_configs).to(device)

# 損失関数と最適化器
criterion = WeightedPlackettLuceLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# 学習ループ
num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    logger.info(f"Epoch {epoch+1}/{num_epochs}")
    
    # 学習
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 検証
    val_loss = validate_epoch(model, val_loader, criterion, device)
    
    # スケジューラーを更新
    scheduler.step()
    
    logger.info(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    
    # ベストモデルの保存
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        # torch.save({
        #     'model_state_dict': model.state_dict(),
        #     'optimizer_state_dict': optimizer.state_dict(),
        #     'epoch': epoch,
        #     'val_loss': val_loss,
        #     'feature_configs': feature_configs
        # }, 'best_model.pth')
        logger.info(f"New best model saved with val_loss: {val_loss:.4f}")

    # 馬券確率の計算・表示（各エポック後）
    print(f"\n{'='*60}")
    print(f"EPOCH {epoch+1} - BETTING PROBABILITIES ANALYSIS")
    print(f"{'='*60}")
    evaluate_betting_probabilities(model, val_loader, device, num_samples=3)
    print(f"{'='*60}\n")

logger.info("Training completed!")